### Objective

To identify a given image is of a cat's or a dog's.

Problem statement can be found here:
https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition

In [56]:
from PIL import Image
import os
from functools import partial
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD, RMSprop
from keras.regularizers import l2
from sklearn import metrics
from sklearn.cross_validation import train_test_split
from scipy import misc
import numpy as np

### Preprocessing the images
creating thumbnails from the original images of size (64x64) so that they're easier to process in a neural network.

In [2]:
def create_thumbnail(filename, dirname, targetdir, size):
    image = Image.open(os.path.join(dirname, filename))
    image.thumbnail(size, Image.ANTIALIAS)
    background = Image.new('RGBA', size, (255,255,255,0)) # Creating a white color background for the image
    background.paste( image, ((size[0]-image.size[0])/2, (size[1]-image.size[1])/2) )
    background.save(os.path.join(targetdir, filename))

In [2]:
size = (64,64)

In [4]:
imgdir = 'train'
thumbdir = 'train-thumbnails'
thumb_create = partial(create_thumbnail, dirname=imgdir, targetdir=thumbdir, size=size)
map(thumb_create, os.listdir(imgdir))
print 'Thumbnails created for training images.'

Thumbnails created for training images.


### Read the training and test images in numpy.array

In [3]:
def load_image(filepath):
    image = misc.imread(filepath)
    return image

In [4]:
train_img_names = os.listdir('train-thumbnails')
test_img_names = os.listdir('test-thumbnails')
all_train = np.array( map(load_image, map(lambda x: os.path.join('train-thumbnails', x), train_img_names) ) )
all_train_labels = map(lambda x: x.split('.', 1)[0], train_img_names)
all_train_labels = map(lambda x: 0 if x=='dog' else 1, all_train_labels)
all_train_labels = np.array(all_train_labels)

In [16]:
train_x, cv_x, train_y, cv_y = train_test_split(all_train, all_train_labels, train_size=0.8)

In [17]:
train_y = np.eye(2)[train_y]

In [26]:
cv_y = np.eye(2)[cv_y]

In [19]:
train_x[0].shape, train_y[0].shape

((64, 64, 3), (2,))

### Creating and training a neural network with the image thumbnails

In [57]:
# Creating a CNN using keras
model = Sequential()
model.add( Convolution2D (
                32, # number of filters
                3, 3, # Convolution size
                border_mode = 'valid',
                input_shape = (64,64,3),
                W_regularizer = l2(0.01)
        ))
model.add( BatchNormalization() )
model.add( Activation('relu') )
model.add( Convolution2D( 32, 3, 3, W_regularizer = l2(0.01) ) )
model.add( BatchNormalization() )
model.add( Activation('relu') )
model.add( MaxPooling2D( pool_size=(2,2) ) )
model.add( Dropout(0.25) )

model.add( Convolution2D( 64, 3, 3, W_regularizer = l2(0.01) ) )
model.add( BatchNormalization() )
model.add( Activation('relu') )
model.add( Convolution2D( 64, 3, 3, W_regularizer = l2(0.01) ) )
model.add( BatchNormalization() )
model.add( Activation('relu') )
model.add( MaxPooling2D( pool_size=(2,2) ) )
model.add( Dropout(0.25) )

model.add( Convolution2D( 128, 3, 3, W_regularizer = l2(0.01)) )
model.add( BatchNormalization() )
model.add( Activation('relu') )
model.add( Convolution2D(128, 3, 3, W_regularizer = l2(0.01)) )
model.add( BatchNormalization() )
model.add( Activation('relu') )
model.add( MaxPooling2D( pool_size=(2,2) ) )
model.add( Dropout(0.25) )

model.add( Flatten() )

model.add( Dense( 128, W_regularizer = l2(0.01) ) )
model.add( Activation('relu') )
model.add( Dropout(0.5) )

model.add( Dense( 2, W_regularizer = l2(0.01) ) )
model.add( Activation('sigmoid') )

sgd = SGD(lr = 1e-3)

model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [58]:
model.fit(train_x, train_y, batch_size=32, nb_epoch=50)

Epoch 1/50
20000/20000 [==============================] - 1367s - loss: 3.2910 - acc: 0.6051  
Epoch 2/50
20000/20000 [==============================] - 1294s - loss: 3.1623 - acc: 0.6719  
Epoch 3/50
20000/20000 [==============================] - 1216s - loss: 3.0559 - acc: 0.7077  
Epoch 4/50
20000/20000 [==============================] - 1255s - loss: 2.9744 - acc: 0.7274  
Epoch 5/50
20000/20000 [==============================] - 1259s - loss: 2.8770 - acc: 0.7510  
Epoch 6/50
20000/20000 [==============================] - 1244s - loss: 2.7948 - acc: 0.7677  
Epoch 7/50
20000/20000 [==============================] - 1272s - loss: 2.7165 - acc: 0.7782  
Epoch 8/50
20000/20000 [==============================] - 1262s - loss: 2.6412 - acc: 0.7925  
Epoch 9/50
20000/20000 [==============================] - 1260s - loss: 2.5671 - acc: 0.7998  
Epoch 10/50
20000/20000 [==============================] - 1272s - loss: 2.5022 - acc: 0.8125  
Epoch 11/50
20000/20000 [========================

In [59]:
loss, accuracy = model.evaluate( cv_x, cv_y )
print 'Accuracy:', accuracy
print 'Loss:', loss

5000/5000 [==============================] - 74s    
Accuracy: 0.8908
Loss: 0.272850845385


# Conclusion

### Achieved almost 90% accuracy on cross validation with Convolutional Neural Network